In [ ]:
%pip install haystack-ai pandas datasets matplotlib python-dotenv qdrant-client langchain-openai langchain_community streamlit sentence-transformers langchain_experimental ollama

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached qdrant_client-1.13.2-py3-none-any.whl.metadata (10 kB)
  Using cached haystack_experimental-0.6.0-py3-none-any.whl.metadata (15 kB)
  Using cached lazy_imports-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached more_itertools-10.6.0-py3-none-any.whl.metadata (37 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     -------------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# sem placa da nvidia
%pip install faiss-cpu

# com placa da nvidia
# %pip install faiss-gpu

   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.7 MB 495.5 kB/s eta 0:00:28
   ---------------------------------------- 0.1/13.7 MB 1.0 MB/s eta 0:00:14
   - -------------------------------------- 0.3/13.7 MB 2.0 MB/s eta 0:00:07
   -- ------------------------------------- 1.0/13.7 MB 4.6 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/13.7 MB 7.9 MB/s eta 0:00:02
   -------------- ------------------------- 4.8/13.7 MB 15.5 MB/s eta 0:00:01
   ------------------------------ --------- 10.3/13.7 MB 38.5 MB/s eta 0:00:01
   ------------------------------------ --- 12.6/13.7 MB 93.0 MB/s eta 0:00:01
   -------------------------------------- - 13.1/13.7 MB 65.6 MB/s eta 0:00:01
   ---------------------------------------  13.4/13.7 MB 54.4 MB/s eta 0:00:01
   ---------------------------------------  13.7/13.7 MB 38.5 MB/s eta 0:00:01



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = api_key  

## dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("rajpurkar/squad_v2")

In [ ]:
dataset

In [ ]:
import pandas as pd

data_union = pd.concat([pd.DataFrame(dataset['train']), pd.DataFrame(dataset['validation'])], axis=0)

## separação de informações

o dataset tem esse formato
- Index(['id', 'title', 'context', 'question', 'answers'], dtype='object')

#### Contexto
existem dois tipos de separação para contexto
- mantendo todas as tuplas separadamente, ou seja, title e seu context correspondente
- unindo todos os context de um mesmo title, reduzindo bastante o numero de tuplas e deixando os contextos mais completos

#### Perguntas e Resposta (Q&A)
- obtendo o titulo, pergunta e resposta de cada tupla e salvando individualmente

In [5]:
from enum import Enum

class Contexto(Enum):
    GERAIS_100 = 'contexto_gerais_100'
    GERAIS_50 = 'contexto_gerais_50'
    GERAIS_25 = 'contexto_gerais_25'
    GERAIS_15 = 'contexto_gerais_15'
    PORTITULO_100 = 'contexto_portitulo_100'
    PORTITULO_50 = 'contexto_portitulo_50'
    PORTITULO_25 = 'contexto_portitulo_25'
    PORTITULO_15 = 'contexto_portitulo_15'
    QA_PAIRS = 'qa_pairs'
    QA_PAIRS_PORTITULO_100 = 'qa_pairs_portitulo_100'
    QA_PAIRS_PORTITULO_50 = 'qa_pairs_portitulo_50'
    QA_PAIRS_PORTITULO_25 = 'qa_pairs_portitulo_25'
    QA_PAIRS_PORTITULO_15 = 'qa_pairs_portitulo_15'
    QA_PAIRS_GERAIS_100 = 'qa_pairs_gerais_100'
    QA_PAIRS_GERAIS_50 = 'qa_pairs_gerais_50'
    QA_PAIRS_GERAIS_25 = 'qa_pairs_gerais_25'
    QA_PAIRS_GERAIS_15 = 'qa_pairs_gerais_15'
    TESTE_PORTITULO = 'contexto_portitulo_teste'
    TESTE_GERAIS = 'contexto_gerais_teste'
    QA_TESTE_PORTITULO = 'qa_teste_portitulo'
    QA_TESTE_GERAIS = 'qa_teste_gerais'

### unindo contextos por titulo

In [ ]:
def consolidate_contexts(group):
    # Remover duplicatas dentro de um mesmo título
    unique_contexts = set(group)
    # Juntar os contextos em um único texto corrido
    return " ".join(unique_contexts)

contextos_df_unindo = data_union.copy().groupby("title")["context"].apply(consolidate_contexts).reset_index()

In [ ]:
# 100% dos dados (embaralhados)
contextos_df_unindo.to_csv(f'''data/{Contexto.PORTITULO_100.value}.csv''', index=False)

# 50% dos dados
contextos_df_unindo.sample(frac=0.5, random_state=42).to_csv(f'''data/{Contexto.PORTITULO_50.value}.csv''', index=False)

# 25% dos dados
contextos_df_unindo.sample(frac=0.25, random_state=42).to_csv(f'''data/{Contexto.PORTITULO_25.value}.csv''', index=False)

# 15% dos dados
contextos_df_unindo.sample(frac=0.15, random_state=42).to_csv(f'''data/{Contexto.PORTITULO_15.value}.csv''', index=False)

print("Arquivos gerados com sucesso!")

### sem unir por titulo

In [ ]:
contextos_df_title = data_union.copy()
contextos_df_title = contextos_df_title[["title", "context"]]

# Embaralha os dados antes de amostrar
contextos_df_title = contextos_df_title.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# 100% dos dados
contextos_df_title.to_csv(f'''data/{Contexto.GERAIS_100.value}.csv''', index=False)

# 50% dos dados
contextos_df_title.sample(frac=0.5, random_state=42).to_csv(f'''data/{Contexto.GERAIS_50.value}.csv''', index=False)

# 25% dos dados
contextos_df_title.sample(frac=0.25, random_state=42).to_csv(f'''data/{Contexto.GERAIS_25.value}.csv''', index=False)

# 15% dos dados
contextos_df_title.sample(frac=0.15, random_state=42).to_csv(f'''data/{Contexto.GERAIS_15.value}.csv''', index=False)

print("Arquivos gerados com sucesso!")

### separando qa

In [ ]:
qa_df = data_union[["title", "question", "answers"]]

def extract_answers(row):
    return row['answers']['text']

qa_df['answers'] = qa_df.apply(extract_answers, axis=1)
qa_df = qa_df[qa_df['answers'].str.len() > 0]

def process_answers(answer_list):
    if not isinstance(answer_list, list):
        return answer_list  # Retorna diretamente caso não seja uma lista
    
    unique_answers = list(set(answer_list))  # Remove duplicatas
    
    if len(unique_answers) == 1:
        return unique_answers[0]  # Retorna como string se houver apenas um item único
    
    return unique_answers  # Retorna a lista se houver múltiplos valores distintos

qa_df['answers'] = qa_df['answers'].apply(process_answers)

qa_df.to_csv(f'''data/qa/{Contexto.QA_PAIRS.value}_geral.csv''', index=False)

## haystack

- inicializando itens do haystack

In [ ]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.converters import TextFileToDocument
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import OpenAIDocumentEmbedder, OpenAITextEmbedder
from haystack.components.writers import DocumentWriter
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
import time

- montando pipeline de indexacao de documentos
- montando pipeline de recuperação e geração

In [ ]:
def pipeline(source_file: str, qa_passado_df: pd.DataFrame, output_filename: str):
    text_file_converter = TextFileToDocument()
    cleaner = DocumentCleaner()
    splitter = DocumentSplitter()
    embedder = OpenAIDocumentEmbedder()
    indexing_pipeline = Pipeline()

    text_embedder = OpenAITextEmbedder()
    template = """Given these contexts, answer the question.
                    Context:
                    {% for doc in documents %}
                        {{ doc.content }}
                    {% endfor %}
                    Question: {{query}}
                    Answer:"""
    prompt_builder = PromptBuilder(template=template)
    llm = OpenAIGenerator()
    rag_pipeline = Pipeline()

    print('------------------- montando pipeline de indexação de docs')
    document_store = InMemoryDocumentStore()
    writer = DocumentWriter(document_store)
    retriever = InMemoryEmbeddingRetriever(document_store)
    
    indexing_pipeline.add_component("converter", text_file_converter)
    indexing_pipeline.add_component("cleaner", cleaner)
    indexing_pipeline.add_component("splitter", splitter)
    indexing_pipeline.add_component("embedder", embedder)
    indexing_pipeline.add_component("writer", writer)

    indexing_pipeline.connect("converter.documents", "cleaner.documents")
    indexing_pipeline.connect("cleaner.documents", "splitter.documents")
    indexing_pipeline.connect("splitter.documents", "embedder.documents")
    indexing_pipeline.connect("embedder.documents", "writer.documents")

    print('------------------- pipeline montada, lendo arquivo')
    indexing_pipeline.run(data={"sources": [f'''data/{source_file}.csv''']})

    print(f"Documentos do arquivo {source_file} indexados com sucesso!")
    print('------------------- montando pipelines de rag (retriver e generator)')

    rag_pipeline.add_component("text_embedder", text_embedder)
    rag_pipeline.add_component("retriever", retriever)
    rag_pipeline.add_component("prompt_builder", prompt_builder)
    rag_pipeline.add_component("llm", llm)

    rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
    rag_pipeline.connect("retriever.documents", "prompt_builder.documents")
    rag_pipeline.connect("prompt_builder", "llm")

    print('------------------- pipelines de rag montada')

    print('------------------- iniciando testes de perguntas e respostas')

    results = []
    i = 0
    total = len(qa_passado_df)
    
    for index, row in qa_passado_df.iterrows():
        query = row["question"]
        expected_answer = row["answers"]
        
        generated_answer = rag_pipeline.run(data={"prompt_builder": {"query": query}, "text_embedder": {"text": query}})

        results.append({
            "title": row["title"],
            "question": query,
            "expected_answer": expected_answer,
            "generated_answer": generated_answer["llm"]["replies"],
        })
        i += 1

        print(f"################### Processando ({i}/{total})...")
        time.sleep(0.1)
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(f'''data/qa/{output_filename}.csv''', index=False)

    print(f"\n-------------------Finalizado! Resultados salvos em {output_filename}")


## resultados haystack por titulo

In [ ]:
qa_df_5_porcento = qa_df.sample(frac=0.05, random_state=42)
qa_df_01_porcento = qa_df.sample(frac=0.001, random_state=42)

In [ ]:
# qa_df_usado = qa_df.copy()
# qa_df_usado.to_csv(f'''data/qa/{Contexto.QA_PAIRS.value}_usado.csv''')
qa_df_usado = pd.read_csv(f'''data/qa/{Contexto.QA_PAIRS.value}_usado.csv''')

In [ ]:
qa_df_usado.head()

In [ ]:
pipeline(Contexto.PORTITULO_15.value, qa_df_usado, Contexto.QA_PAIRS_PORTITULO_15.value)

In [ ]:
pipeline(Contexto.PORTITULO_25.value, qa_df_usado, Contexto.QA_PAIRS_PORTITULO_25.value)

In [ ]:
pipeline(Contexto.PORTITULO_50.value, qa_df_usado, Contexto.QA_PAIRS_PORTITULO_50.value)

In [ ]:
pipeline(Contexto.PORTITULO_100.value, qa_df_usado, Contexto.QA_PAIRS_PORTITULO_100.value)

## resultados haystack geral

In [ ]:
pipeline(Contexto.GERAIS_15.value, qa_df_usado, Contexto.QA_PAIRS_GERAIS_15.value)

In [ ]:
pipeline(Contexto.GERAIS_25.value, qa_df_usado, Contexto.QA_PAIRS_GERAIS_25.value)

In [ ]:
pipeline(Contexto.GERAIS_50.value, qa_df_usado, Contexto.QA_PAIRS_GERAIS_50.value)

In [ ]:
pipeline(Contexto.GERAIS_100.value, qa_df_usado, Contexto.QA_PAIRS_GERAIS_100.value)

## comparação de respostas com llm

In [ ]:
from openai import OpenAI
import re

def use_llm(esperada, obtida):
    cliente = OpenAI(api_key=api_key)

    prompt = f'''
        Você é um assistente que compara textos. 
        Compare a resposta esperada '{esperada}' com a resposta adquirida '{obtida}' e diga se são semanticamente semelhantes. 

        Responda em uma escala de 0 a 10, onde 0 significa que os textos são completamente diferentes e 10 significa que são idênticos em sentido e semanticamente.

        Tente entender o sentido completo da resposta, não apenas palavras-chave.
        Mas também se atente a detalhes como palavras-chave e seus significados.
        Por exemplo, orações com palavras diferentes mas com o mesmo significado devem ser consideradas semelhantes.
        Em caso de datas, números ou informações específicas, considere a semelhança do contexto em que estão inseridos.

        Importante: Uma resposta adquirida pode ser mais longa e detalhada que a resposta esperada. Isso não necessariamente a torna diferente, desde que ela ainda aborde o mesmo tópico e não contradiga a resposta esperada. Portanto, mesmo que a resposta adquirida seja mais longa, se ela ainda estiver alinhada com a resposta esperada, considere-a semelhante.

        Responda apenas o número equivalente à semelhança dos textos.

        # exemplo de saida
        10
    '''

    messages = [
        {
            "role": "user",
            "content": prompt
        },
    ]

    response = cliente.chat.completions.create(
        model="gpt-4o",
        store=True,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

def remove_specific_characters(input_string):
    input_string = re.sub(r'(?<=[a-zA-Z])\.', '', input_string)
    input_string = re.sub(r'[\[\]\'\"]', '', input_string)
    return input_string

def is_similar_using_llm(esperada: str = "",obtida: str = ""):
    if isinstance(esperada, list):
        esperada = esperada[0]
    if isinstance(obtida, list):
        obtida = obtida[0]

    esperada = remove_specific_characters(esperada)
    obtida = remove_specific_characters(obtida)
    response = use_llm(esperada, obtida)
    return response


In [ ]:
teste = is_similar_using_llm("archdioceses or departments of the Roman Curia", "The Roman Curia.")
teste2 = is_similar_using_llm('Central Standard Time',"['The contexts provided do not mention Saskatoon or its time observance specific to time zones, Daylight Saving Time, or any other relevant information. Therefore, based on the information given, I cannot determine what time Saskatoon observes all year long.']")
teste3 = is_similar_using_llm('VHF omnidirectional range',['VOR stands for VHF Omnidirectional Range.'])
teste4 = is_similar_using_llm('32nd',"['In 2009, Tucson ranked as the 32nd largest city in the United States.']")
teste5 = is_similar_using_llm('1996',"['Labour published a new draft manifesto in 1996, called ""New Labour, New Life For Britain.""']")
print(f"Teste 1: {teste}, Teste 2: {teste2}, Teste 3: {teste3}, Teste 4 que devia dar 10: {teste4}, Teste 5 que devia dar 10: {teste5}")

In [ ]:
frase1="email, web-hosting, or online storage services"
frase2="['Internet hosting services provide the infrastructure and technology needed to make websites accessible on the Internet. This includes server space for storing website files, bandwidth for transmitting data to users, domain name registration, email accounts, and often additional services such as database management, security features, and technical support. Hosting services enable individuals and organizations to have an online presence by hosting their web content and applications.']"
print(is_similar_using_llm(frase1, frase2))

In [ ]:
def compare_respostas_usando_llm(arquivo_csv: str, qa='qa'):
    df = pd.read_csv(f'''data/{qa}/{arquivo_csv}.csv''')
    df['is_similar'] = df.apply(lambda row: is_similar_using_llm(row['expected_answer'], row['generated_answer']), axis=1)
    df.to_csv(f'''data/{qa}/{arquivo_csv}.csv''', index=False)

def compare_respostas_usando_llm_contagem(arquivo_csv: str, qa='qa'):
    df = pd.read_csv(f'''data/{qa}/{arquivo_csv}.csv''')

    count = 0
    total = len(df)

    for index, row in df.iterrows():
        df.loc[index, 'is_similar'] = is_similar_using_llm(row['expected_answer'], row['generated_answer'])
        
        count += 1
        
        print(f'*********** Progresso: {count}/{total}')

    df.to_csv(f'''data/{qa}/{arquivo_csv}.csv''', index=False)

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_GERAIS_15.value)

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_GERAIS_25.value)

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_GERAIS_50.value)

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_GERAIS_100.value)

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_PORTITULO_15.value)

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_PORTITULO_25.value)

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_PORTITULO_50.value)

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_PORTITULO_100.value)

## adicionando testes a parte

In [ ]:
contextos_df_title.sample(frac=0.05, random_state=42).to_csv(f'''data/{Contexto.TESTE_PORTITULO.value}.csv''', index=False)
contextos_df_unindo.sample(frac=0.05, random_state=42).to_csv(f'''data/{Contexto.TESTE_GERAIS.value}.csv''', index=False)

In [ ]:
qa_df_novo = qa_df.sample(frac=0.01, random_state=42)

In [ ]:
pipeline(Contexto.TESTE_PORTITULO.value, qa_df_novo, Contexto.QA_TESTE_PORTITULO.value)

In [ ]:
pipeline(Contexto.TESTE_GERAIS.value, qa_df_novo, Contexto.QA_TESTE_PORTITULO.value)

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_TESTE_PORTITULO.value)

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_TESTE_GERAIS.value)

## avaliando resultados

In [ ]:
# base que quero avaliar
QA_avaliado = Contexto.QA_PAIRS_GERAIS_100.value

In [ ]:
obtendo_respostas = pd.read_csv(f'''data/qa/{QA_avaliado}.csv''')

In [ ]:
# Criar um dicionário para armazenar a contagem por faixa de 0 a 10
bins = list(range(11))  # Criando bins de 0 a 10

# Contar as ocorrências em cada faixa
contagem = obtendo_respostas["is_similar"].value_counts().sort_index()
contagem = contagem.reindex(bins, fill_value=0)  # Garantir que todas as faixas apareçam

# Calcular porcentagem
total = contagem.sum()
porcentagem = (contagem / total) * 100


# Exibir a tabela de porcentagens
print(f'Distribuição de similaridade no dataset {QA_avaliado} (%):')
print(porcentagem)

In [ ]:
import matplotlib.pyplot as plt

cores = ["blue", "red", "green", "purple", "orange", "brown", "pink", "gray", "cyan", "magenta"]

# Criando um gráfico de barras
plt.figure(figsize=(6, 4))
obtendo_respostas["is_similar"].value_counts().plot(kind="bar", color=cores)
plt.title(f'''Distribuição de similaridade {QA_avaliado}''')
plt.xlabel("o quão similar é")
plt.ylabel("Frequência")
plt.xticks(rotation=0)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

# Criando um gráfico de pizza
plt.figure(figsize=(6, 6))
obtendo_respostas["is_similar"].value_counts().plot(kind="pie", autopct="%1.1f%%", colors=cores, startangle=90, wedgeprops={"edgecolor": "black"})
plt.title(f'''Proporção de similaridade {QA_avaliado}''')
plt.ylabel("")  # Removendo label desnecessário
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Dados das distribuições de similaridade
datasets = {
    "qa_pairs_portitulo_15": [47.18, 0.06, 6.73, 15.52, 0.88, 2.74, 0.90, 11.26, 11.54, 2.44, 0.76],
    "qa_pairs_portitulo_25": [42.39, 0.00, 6.93, 15.67, 0.80, 3.04, 0.96, 12.90, 13.68, 2.84, 0.80],
    "qa_pairs_portitulo_50": [34.17, 0.04, 6.43, 16.33, 1.04, 4.01, 0.84, 15.73, 16.37, 3.57, 1.46],
    "qa_pairs_portitulo_100": [16.95, 0.00, 5.55, 18.89, 1.24, 5.89, 0.78, 20.83, 22.38, 5.25, 2.24],
    "qa_pairs_gerais_15": [29.47, 0.06, 6.81, 18.13, 0.76, 4.35, 0.76, 16.43, 17.47, 3.99, 1.76],
    "qa_pairs_gerais_25": [25.32, 0.02, 6.73, 17.97, 1.06, 5.09, 0.72, 17.33, 18.81, 5.03, 1.92],
}

# Configuração dos gráficos
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(datasets["qa_pairs_portitulo_15"]))  # Posições no eixo X
width = 0.15  # Largura das barras

# Cores para os datasets
colors = ['b', 'g', 'r', 'c', 'm', 'y']

# Criando barras para cada dataset
for i, (label, values) in enumerate(datasets.items()):
    ax.bar(x + i * width, values, width, label=label, color=colors[i])

# Ajustes no gráfico
ax.set_xlabel("Níveis de Similaridade")
ax.set_ylabel("Percentual (%)")
ax.set_title("Comparação das Distribuições de Similaridade")
ax.set_xticks(x + width)
ax.set_xticklabels(range(11))
ax.legend()
plt.xticks(rotation=45)

# Exibir gráfico
plt.show()


<hr>

## langchain

In [ ]:
import pandas as pd

In [39]:
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain.schema import (
    SystemMessage,
    HumanMessage
)
import time

In [ ]:
def pipeline_langchain_openai(base: str, qa_passado_df, output_filename: str):

    data_df = pd.read_csv(f'''data/{base}.csv''')

    chat = ChatOpenAI(
        model="gpt-4o",
    )
    loader = DataFrameLoader(data_df, page_content_column="context")
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    documents = loader.load()

    print('------------------- obtendo documentos para pipeline')
    qdrant = Qdrant.from_documents(
        documents=documents,
        embedding=embeddings,
        location=":memory:",
        collection_name="rag"
    )

    print('------------------- finalizando documentos para pipeline')

    print('------------------- iniciando testes de perguntas e respostas')

    results = []
    i = 0
    total = len(qa_passado_df)
    
    for index, row in qa_passado_df.iterrows():
        query = row["question"]
        expected_answer = row["answers"]
        
        results_retriver = qdrant.similarity_search(query, k=3)
        source_knowledge = "\n".join([x.page_content for x in results_retriver])
        augment_prompt = f"""Use the context below to answer the question.

        Context:
        {source_knowledge}
        -------------------------
        Succinct answers are necessary, always focusing on answering the question. 
        To get straight to the point, avoid unnecessary information.
        Avoid repeat the question in the answer and focus in the objective answer.
        -------------------------
        Question: {query}"""


        messages = [
            SystemMessage(content="You are a RAG that uses contexts to answer questions."),
            HumanMessage(content=augment_prompt)
        ]

        res = chat.invoke(messages)

        results.append({
            "title": row["title"],
            "question": query,
            "expected_answer": expected_answer,
            "generated_answer": res.content,
        })
        i += 1

        print(f"################### Processando ({i}/{total})...")
        time.sleep(0.1)
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(f'''data/qa_langchain_openai/{output_filename}.csv''', index=False)

    print(f"\n-------------------Finalizado! Resultados salvos em {output_filename}")

- dataset utilizado na outra 

In [ ]:
qa_df_usado = pd.read_csv(f'''data/qa/{Contexto.QA_PAIRS.value}_usado.csv''')

aplicando em contextos

In [ ]:
pipeline_langchain_openai(Contexto.GERAIS_100.value, qa_df_usado, Contexto.QA_PAIRS_GERAIS_100.value)

In [ ]:
pipeline_langchain_openai(Contexto.GERAIS_50.value, qa_df_usado, Contexto.QA_PAIRS_GERAIS_50.value)

In [ ]:
pipeline_langchain_openai(Contexto.GERAIS_25.value, qa_df_usado, Contexto.QA_PAIRS_GERAIS_25.value)

In [ ]:
pipeline_langchain_openai(Contexto.GERAIS_15.value, qa_df_usado, Contexto.QA_PAIRS_GERAIS_15.value)

In [ ]:
pipeline_langchain_openai(Contexto.PORTITULO_15.value, qa_df_usado, Contexto.QA_PAIRS_PORTITULO_15.value)

In [ ]:
pipeline_langchain_openai(Contexto.PORTITULO_25.value, qa_df_usado, Contexto.QA_PAIRS_PORTITULO_25.value)

In [ ]:
pipeline_langchain_openai(Contexto.PORTITULO_50.value, qa_df_usado, Contexto.QA_PAIRS_PORTITULO_50.value)

In [ ]:
pipeline_langchain_openai(Contexto.PORTITULO_100.value, qa_df_usado, Contexto.QA_PAIRS_PORTITULO_100.value)

- avaliacao

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_GERAIS_100.value, 'qa_langchain_openai')

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_GERAIS_50.value, 'qa_langchain_openai')

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_GERAIS_25.value, 'qa_langchain_openai')

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_GERAIS_15.value, 'qa_langchain_openai')

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_PORTITULO_100.value, 'qa_langchain_openai')

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_PORTITULO_50.value, 'qa_langchain_openai')

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_PORTITULO_25.value, 'qa_langchain_openai')

In [ ]:
compare_respostas_usando_llm_contagem(Contexto.QA_PAIRS_PORTITULO_15.value, 'qa_langchain_openai')

<hr>

## rag deepseek

In [72]:
from langchain_community.document_loaders import DataFrameLoader 
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama 
import pandas as pd
import re

In [ ]:
## rode no seu terminal
## ollama run deepseek-r1:8b

In [75]:
# qa_df_usado = pd.read_csv(f'''data/qa/{Contexto.QA_PAIRS.value}_usado.csv''')
qa_df = pd.read_csv(f'''data/qa/{Contexto.QA_PAIRS.value}_usado.csv''')

In [80]:
qa_df_usado = qa_df.sample(frac=0.005, random_state=42)

In [74]:
def pipeline_langchain_deepseek(base: str, qa_passado_df, output_filename: str):

    data_df = pd.read_csv(f'''data/{base}.csv''')

    llm = Ollama(model="deepseek-r1:8b") 

    loader = DataFrameLoader(data_df, page_content_column="context")
    documents = loader.load()
    embeddings_model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name) 

    print('------------------- obtendo documentos para pipeline')
    qdrant = Qdrant.from_documents(
        documents=documents,
        embedding=embeddings,
        location=":memory:",
        collection_name="rag"
    )

    print('------------------- finalizando documentos para pipeline')

    print('------------------- iniciando testes de perguntas e respostas')

    results = []
    i = 0
    total = len(qa_passado_df)
    
    for index, row in qa_passado_df.iterrows():
        query = row["question"]
        expected_answer = row["answers"]
        
        results_retriver = qdrant.similarity_search(query, k=3)
        source_knowledge = "\n".join([x.page_content for x in results_retriver])
        augment_prompt = f"""Use the context below to answer the question.

        Context:
        {source_knowledge}
        -------------------------
        1. Use ONLY the context below.  
        2. If in doubt, say "I don't know".
        -------------------------
        Succinct answers are necessary, always focusing on answering the question. 
        To get straight to the point, avoid unnecessary information.
        Avoid repeat the question in the answer and focus in the objective answer.
        -------------------------
        Question: {query}"""


        messages = [
            SystemMessage(content="You are a RAG that uses contexts to answer questions."),
            HumanMessage(content=augment_prompt)
        ]

        resp = llm.invoke(messages)
        response = re.sub(r"<think>.*?</think>\n?", "",resp, flags=re.DOTALL)

        results.append({
            "title": row["title"],
            "question": query,
            "expected_answer": expected_answer,
            "generated_answer": response,
        })
        i += 1

        print(f"################### Processando ({i}/{total})...")
        time.sleep(0.1)
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(f'''data/qa_langchain_deepseek/{output_filename}.csv''', index=False)

    print(f"\n-------------------Finalizado! Resultados salvos em {output_filename}")

In [ ]:
pipeline_langchain_deepseek(Contexto.PORTITULO_15.value, qa_df_usado, Contexto.QA_TESTE_PORTITULO.value)

------------------- obtendo documentos para pipeline


c:\Users\nessa\OneDrive\Desktop\coding\machineLearning\.venv\Lib\site-packages\langchain_community\vectorstores\qdrant.py:192: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 20032 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  self.client.upsert(


------------------- finalizando documentos para pipeline
------------------- iniciando testes de perguntas e respostas
################### Processando (1/25)...
################### Processando (2/25)...
################### Processando (3/25)...
################### Processando (4/25)...
################### Processando (5/25)...
################### Processando (6/25)...
################### Processando (7/25)...
################### Processando (8/25)...
################### Processando (9/25)...
################### Processando (10/25)...


In [ ]:
import ollama
import re

def use_llm(esperada, obtida):
    prompt = f'''
        Você é um assistente que compara textos. 
        Compare a resposta esperada '{esperada}' com a resposta adquirida '{obtida}' e diga se são semanticamente semelhantes. 

        Responda em uma escala de 0 a 10, onde 0 significa que os textos são completamente diferentes e 10 significa que são idênticos em sentido e semanticamente.

        Tente entender o sentido completo da resposta, não apenas palavras-chave.
        Mas também se atente a detalhes como palavras-chave e seus significados.
        Por exemplo, orações com palavras diferentes mas com o mesmo significado devem ser consideradas semelhantes.
        Em caso de datas, números ou informações específicas, considere a semelhança do contexto em que estão inseridos.

        Importante: Uma resposta adquirida pode ser mais longa e detalhada que a resposta esperada. Isso não necessariamente a torna diferente, desde que ela ainda aborde o mesmo tópico e não contradiga a resposta esperada. Portanto, mesmo que a resposta adquirida seja mais longa, se ela ainda estiver alinhada com a resposta esperada, considere-a semelhante.

        Responda apenas o número equivalente à semelhança dos textos.

        # exemplo de saída
        10
    '''
    
    messages = [{"role": "user", "content": prompt}]
    response = ollama.chat(model='deepseek-r1:8b', messages=messages)
    return response['message']['content'].strip()

def remove_specific_characters(input_string):
    input_string = re.sub(r'(?<=[a-zA-Z])\.', '', input_string)
    input_string = re.sub(r'[\[\]\'\"]', '', input_string)
    return input_string

def is_similar_using_llm(esperada: str = "", obtida: str = ""):
    if isinstance(esperada, list):
        esperada = esperada[0]
    if isinstance(obtida, list):
        obtida = obtida[0]

    esperada = remove_specific_characters(esperada)
    obtida = remove_specific_characters(obtida)
    response = use_llm(esperada, obtida)
    return response
